In [9]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv(
    'data/export_table_log.csv',
    quotechar='"',
    encoding='utf-8',
    on_bad_lines='skip'  # pandas >= 1.3
)

# Ép kiểu timecreated nếu cần
df['timecreated'] = pd.to_numeric(df['timecreated'], errors='coerce')

# Lọc các dòng liên quan đến lesson
df = df[df['action'].isin(['started', 'resumed', 'ended']) & (df['target'] == 'lesson')]

# Sắp xếp để xử lý theo thời gian
df = df.sort_values(by=['userid', 'objectid', 'timecreated'])

# Danh sách kết quả hoàn thành
results = []
incomplete_sessions = []

# Duyệt từng nhóm userid + lessonid
for (userid, lessonid), group in df.groupby(['userid', 'objectid']):
    print(f"\n📘 User {userid} - Lesson {lessonid}")
    session_start = None
    total_duration = 0
    ended = False

    for _, row in group.iterrows():
        action = row['action']
        time = row['timecreated']

        if pd.isna(time):
            continue

        if action == 'started':
            print(f"  🔹 STARTED at {time}")
            session_start = time
            ended = False

        elif action == 'resumed':
            if session_start is None:
                print(f"  🔸 RESUMED (no start) at {time}")
                session_start = time
            else:
                print(f"  🔸 RESUMED (continued) at {time}")
            ended = False

        elif action == 'ended':
            if session_start is not None:
                duration = time - session_start
                print(f"  ✅ ENDED at {time}, duration = {duration} sec")
                total_duration += duration
                session_start = None
                ended = True
            else:
                print(f"  ⚠️  ENDED but no session_start")

    # Sau khi duyệt xong từng nhóm
    if ended:
        print(f"  ✅✅ TOTAL TIME: {total_duration} seconds")
        results.append({
            'userid': userid,
            'lessonid': lessonid,
            'total_duration_seconds': total_duration
        })
    elif session_start is not None:
        # Ước lượng thời gian nếu không có ended
        last_time = group['timecreated'].max()
        estimated_duration = last_time - session_start
        print(f"  ⏳ No 'ended', estimating duration = {estimated_duration} sec (last activity at {last_time})")
        results.append({
            'userid': userid,
            'lessonid': lessonid,
            'total_duration_seconds': estimated_duration,
            'estimated': True
        })
    else:
        print(f"  ⛔ NO ENDED and NO STARTED session, session ignored")
        incomplete_sessions.append({
            'userid': userid,
            'lessonid': lessonid,
            'note': 'no started or ended'
        })

# Xuất kết quả hoàn thành
result_df = pd.DataFrame(results)
result_df.to_csv('lesson_duration_result.csv', index=False)
print("✅ Đã xuất kết quả hoàn thành ra lesson_duration_result.csv")
print(result_df)

# Xuất kết quả chưa hoàn thành (nếu có)
if incomplete_sessions:
    pd.DataFrame(incomplete_sessions).to_csv('incomplete_sessions.csv', index=False)
    print("⚠️ Có bài học chưa kết thúc (hoặc không thể tính), lưu vào: incomplete_sessions.csv")


📘 User 4 - Lesson 17.0
  🔹 STARTED at 1753193951
  🔸 RESUMED (continued) at 1753194414
  ✅ ENDED at 1753194424, duration = 473 sec
  ✅✅ TOTAL TIME: 473 seconds
✅ Đã xuất kết quả hoàn thành ra lesson_duration_result.csv
   userid  lessonid  total_duration_seconds
0       4      17.0                     473
